# View table for dashboard

## Calendar table for dashboard

In [0]:
%sql
CREATE OR REPLACE TABLE anac_aeronautical_occurrences_in_brazilian_civil_aviation.gold_layer.dim_calendar AS
WITH dates AS (
  SELECT explode(sequence(to_date('2007-01-01'), to_date('2025-12-31'), interval 1 day)) AS Date
)
SELECT
    Date,
    year(Date) AS Ano,
    month(Date) AS Mes,
    day(Date) AS Dia,
    quarter(Date) AS Trimestre,
    date_format(Date, 'MMMM') AS NomeMes,
    date_format(Date, 'EEE') AS NomeDiaAbrev,
    date_format(Date, 'EEEE') AS NomeDia,
    lpad(month(Date), 2, '0') AS MesNumPad,
    concat(year(Date), '-', lpad(month(Date), 2, '0')) AS AnoMes,
    CASE WHEN dayofweek(Date) IN (1,7) THEN 1 ELSE 0 END AS IsWeekend
FROM dates;


## Imports

In [0]:
df_aircraft = spark.read.table("anac_aeronautical_occurrences_in_brazilian_civil_aviation.gold_layer.gov_aircraft_dim")
df_factor = spark.read.table("anac_aeronautical_occurrences_in_brazilian_civil_aviation.gold_layer.gov_significant_factor_dim")
df_occurrence = spark.read.table("anac_aeronautical_occurrences_in_brazilian_civil_aviation.gold_layer.gov_occurrence_fact")
df_recommendation = spark.read.table("anac_aeronautical_occurrences_in_brazilian_civil_aviation.gold_layer.gov_recommendation_dim")
df_type = spark.read.table("anac_aeronautical_occurrences_in_brazilian_civil_aviation.gold_layer.gov_type_occurrence_dim")
df_calendar = spark.read.table("anac_aeronautical_occurrences_in_brazilian_civil_aviation.gold_layer.dim_calendar")

## View table

In [0]:
%sql
CREATE OR REPLACE VIEW anac_aeronautical_occurrences_in_brazilian_civil_aviation.gold_layer.vw_anac_dashboard AS
SELECT
  ac.aeronave_matricula,
  ac.aeronave_fabricante,
  ac.aeronave_modelo,
  ac.aeronave_tipo_veiculo,
  ac.aeronave_motor_tipo,
  ac.aeronave_motor_quantidade,
  ac.aeronave_assentos,
  ac.aeronave_ano_fabricacao,
  ac.aeronave_voo_origem,
  ac.aeronave_voo_destino,
  ac.aeronave_fase_operacao,
  ac.aeronave_nivel_dano,
  ac.aeronave_fatalidades_total,
  tp.ocorrencia_tipo, 
  tp.ocorrencia_tipo_categoria,
  fc.fatores_concatenados,
  fc.aspectos_concatenados,
  fc.condicionantes_concatenados,
  fc.areas_concatenadas,
  oc.ocorrencia_classificacao,
  oc.ocorrencia_cidade,
  oc.ocorrencia_uf,
  oc.ocorrencia_pais,
  oc.ocorrencia_dia,
  oc.ocorrencia_hora,
  oc.total_aeronaves_envolvidas
FROM anac_aeronautical_occurrences_in_brazilian_civil_aviation.gold_layer.gov_occurrence_fact oc
LEFT JOIN anac_aeronautical_occurrences_in_brazilian_civil_aviation.gold_layer.gov_aircraft_dim ac
  ON oc.id_dim_aeronave = ac.id_dim_aeronave
LEFT JOIN anac_aeronautical_occurrences_in_brazilian_civil_aviation.gold_layer.gov_type_occurrence_dim tp
  ON oc.id_dim_ocorrencia_tipo = tp.id_dim_ocorrencia_tipo
LEFT JOIN anac_aeronautical_occurrences_in_brazilian_civil_aviation.gold_layer.gov_significant_factor_dim fc
  ON oc.id_dim_fator = fc.id_dim_fator
